In [1]:
import utils

utils.widen_ipython_window()

import pandas as pd
import datetime
import numpy as np
import gc
import tensorflow as tf
from keras import backend as K 



2023-01-07 23:12:45.286417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 23:12:45.815944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-01-07 23:12:45.816007: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-01-07 23:12:45.81

In [2]:
#
# Paths
#

MAIN_PATH = "/home/mahesh/Desktop/ML/kaggle/amex/"

# Data
PATH_TO_DATA                = MAIN_PATH + "data/"
PATH_TO_PROCESSED_DATA      = PATH_TO_DATA + "processed/"
PATH_TO_PROCESSED2_DATA     = PATH_TO_DATA + "processed2/"
PATH_TO_PROCESSED4_DATA     = PATH_TO_DATA + "processed4/"

PATH_TO_GRU_NAN_EMBEDDINGS_DATA = PATH_TO_DATA + "gru_nan_embeddings_full/"

FILENAME_TRAIN_DATA_CSV     = PATH_TO_DATA + "orig/train_data.csv"
FILENAME_TRAIN_LABELS_CSV   = PATH_TO_DATA + "orig/train_labels.csv"
FILENAME_TEST_DATA_CSV      = PATH_TO_DATA + "orig/test_data.csv"
FILENAME_SAMPLE_SUBMISSION_CSV = PATH_TO_DATA + "orig/sample_submission.csv"

FILENAME_TRAIN_DATA_FEATHER = PATH_TO_PROCESSED_DATA + "train_data.f"
FILENAME_TRAIN_PROCESSED2_DATA_FEATHER   = PATH_TO_PROCESSED2_DATA + "train_data.f"
FILENAME_TRAIN_PROCESSED2_LABELS_FEATHER = PATH_TO_PROCESSED2_DATA + "train_labels.f"
FILENAME_TRAIN_PROCESSED2_DATA_CAT_NOCHANGE_FEATHER   = PATH_TO_PROCESSED2_DATA + "train_data_cat_nochange.f"

FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_FEATHER = PATH_TO_GRU_NAN_EMBEDDINGS_DATA + "train_data.f"
FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_NORMALIZED_FEATHER = PATH_TO_GRU_NAN_EMBEDDINGS_DATA + "train_data_normalized.f"


FILENAME_TEST_CUSTOMER_HASHES  = PATH_TO_PROCESSED2_DATA + "test_customer_hashes_data.pq"
FILENAME_TEST_HASH_DATA        = PATH_TO_PROCESSED2_DATA + "test_hashes_data"
FILENAME_GRU_SUBMISSION        = PATH_TO_PROCESSED2_DATA + "submission_gru.csv"

FILENAME_TRAIN_PROCESSED4_FE_DATA_RNN_FEATHER = PATH_TO_PROCESSED4_DATA + "train_FE_data_RNN.f"

# Models
PATH_TO_MODEL   = PATH_TO_GRU_NAN_EMBEDDINGS_DATA

    1. Feature engineer the data
    2. Iterate over the folds
        a. Load the customer IDs corresponding to the train and val data
        b. Extract the data and labels corresponding to the customer IDs

In [4]:
RUN_FEATURE_ENGINEERING = 1

def feature_engineer(train, PAD_CUSTOMER_TO_13_ROWS = True, targets = None, edit_cid_time = False, num_normalized = False):
        
    # REDUCE STRING COLUMNS 
    # from 64 bytes to 8 bytes, and 10 bytes to 3 bytes respectively
    #train['customer_ID'] = train['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    if edit_cid_time:
        train['customer_ID'] = train['customer_ID'].str[-16:].apply(lambda x:int(x,16)).astype('int64')
        train.S_2 = pd.to_datetime( train.S_2 )
    # REDUCE STRING COLUMNS 
    # from 64 bytes to 8 bytes, and 10 bytes to 3 bytes respectively
    train['year'] = (train.S_2.dt.year-2000).astype('int8')
    train['month'] = (train.S_2.dt.month).astype('int8')
    train['day'] = (train.S_2.dt.day).astype('int8')
    del train['S_2']
        
    # LABEL ENCODE CAT COLUMNS (and reduce to 1 byte)
    # with 0: padding, 1: nan, 2,3,4,etc: values
    d_63_map = {'CL':1, 'CO':2, 'CR':3, 'XL':4, 'XM':5, 'XZ':6}
    train['D_63'] = train.D_63.map(d_63_map).fillna(0).astype('int8')

    d_64_map = {'-1':1,'O':2, 'R':3, 'U':4}
    train['D_64'] = train.D_64.map(d_64_map).fillna(0).astype('int8')
    
    CATS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
    OFFSETS = [1,0,1,1,2,1,2,1,1] #2 minus minimal value in full train csv
    # then 0 will be padding, 1 will be NAN, 2,3,4,etc will be values
    for c,s in zip(CATS,OFFSETS):
        train[c] = train[c] + s
        train[c] = train[c].fillna(0).astype('int8')
    CATS += ['D_63','D_64']
    
    # ADD NEW FEATURES HERE
    # EXAMPLE: train['feature_189'] = etc etc etc
    # EXAMPLE: train['feature_190'] = etc etc etc
    # IF CATEGORICAL, THEN ADD TO CATS WITH: CATS += ['feaure_190'] etc etc etc
    
    # REDUCE MEMORY DTYPE
    SKIP = ['customer_ID','year','month','day']
    for c in train.columns:
        if c in SKIP: continue
        if str( train[c].dtype )=='int64':
            train[c] = train[c].astype('int32')
        if str( train[c].dtype )=='float64':
            train[c] = train[c].astype('float32')

    #
    # For numerical features that have NaNs add as extra column that indicates if 
    # the value is not-nan (1) or nan. Then set the nans to zero.
    #
    NUMS_NO_E = [] # We don't have numerical features without embeddings because we need to pad these features with zeros
    NUMS_WITH_E = []
    zeros = np.zeros((train.shape[0],), dtype='int8')
    ones = np.ones((train.shape[0],), dtype='int8')
    for c in train.columns:
        if ((c in SKIP) or (c in CATS)): continue
        c_exists = c + "_exists"
        train[c_exists] = np.where(np.invert(np.isnan(train[c].values)), ones, zeros)
        #if num_normalized: train[c]=(train[c]-train[c].min())/(train[c].max()-train[c].min())
        if num_normalized: train[c]=(train[c]-train[c].mean())/(train[c].std(ddof=0))
        train[c] = train[c].fillna(0)
        NUMS_WITH_E.append(c)
        NUMS_WITH_E.append(c_exists)
    #
    # Pad data for customers who don't have 13 months of data
    #
    tmp = train[['customer_ID']].groupby('customer_ID').customer_ID.agg('count')
    more = np.array([],dtype='int64') 
    for j in range(1,13):
        i = tmp.loc[tmp==j].index.values
        #print(i)
        more = np.concatenate([more,np.repeat(i,13-j)])
        #print(more)
    df = train.iloc[:len(more)].copy().fillna(0)
    df = df * 0
    # df = df * 0 - 1 #pad numerical columns with -1
    df[CATS] = df[CATS].astype('int8') #pad categorical columns with 0
    df['customer_ID'] = more
    train = pd.concat([train,df],axis=0,ignore_index=True)

        
    # ADD TARGETS (and reduce to 1 byte)
    if targets is not None:
        train = train.merge(targets,on='customer_ID',how='left')
        train.target = train.target.astype('int8')
        
    # FILL NAN
    #train = train.fillna(-0.5) #this applies to numerical columns
    
    # SORT BY CUSTOMER THEN DATE
    train = train.sort_values(['customer_ID','year','month','day']).reset_index(drop=True)
    train = train.drop(['year','month','day'],axis=1)
    
    # REARRANGE COLUMNS WITH 11 CATS FIRST
    COLS = list(train.columns[1:])
    #COLS = ['customer_ID'] + CATS + [c for c in COLS if c not in CATS]
    COLS = ['customer_ID'] + CATS + NUMS_NO_E + NUMS_WITH_E + ['target']
    train = train[COLS]
    
    num_features = len(CATS + NUMS_NO_E + NUMS_WITH_E)
    
    utils.pt(f'NUM_FEATURES : {num_features}, CATS : {len(CATS)}, NUMS_NO_E : {len(NUMS_NO_E)}, NUMS_WITH_E : {len(NUMS_WITH_E)}' )
    
    return train

def read_train_data():
    # Load the data
    train_full_data   = pd.read_feather(FILENAME_TRAIN_PROCESSED2_DATA_CAT_NOCHANGE_FEATHER)
    train_full_labels = pd.read_feather(FILENAME_TRAIN_PROCESSED2_LABELS_FEATHER)
    utils.pt("Reading raw data")
    train_full_data.info(memory_usage="deep")
    return (train_full_data, train_full_labels)

def feature_engineer_full_data_and_save_to_file(train_data_output_filename, num_normalized = False):
    (train_full_data, train_full_labels) = read_train_data()
    
    # Feature engineer
    utils.pt("Starting feature engineering")
    train_FE_data = feature_engineer(train_full_data, PAD_CUSTOMER_TO_13_ROWS = True, 
                                     targets = train_full_labels, num_normalized = num_normalized)
    utils.pt("Completed feature engineering")
    
    train_FE_data.info(memory_usage="deep")
    
    utils.pt("Writing feature engineered data to disk")
    train_FE_data.to_feather(train_data_output_filename)
    
    utils.gc_l([train_full_data, train_full_labels, train_FE_data])
    
if RUN_FEATURE_ENGINEERING:
    #feature_engineer_full_data_and_save_to_file(train_data_output_filename = FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_FEATHER, num_normalized = False)
    feature_engineer_full_data_and_save_to_file(train_data_output_filename = FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_NORMALIZED_FEATHER
                                                , num_normalized = True)

2022-09-20 20:46:51.915774 : Reading raw data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: datetime64[ns](1), float32(176), float64(9), int32(1), int64(1), object(2)
memory usage: 4.7 GB
2022-09-20 20:46:52.464342 : Starting feature engineering
2022-09-20 20:47:22.159561 : NUM_FEATURES : 365, CATS : 11, NUMS_NO_E : 0, NUMS_WITH_E : 354
2022-09-20 20:47:22.160803 : Completed feature engineering
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5965869 entries, 0 to 5965868
Columns: 367 entries, customer_ID to target
dtypes: float32(176), float64(1), int64(1), int8(189)
memory usage: 5.1 GB
2022-09-20 20:47:22.171760 : Writing feature engineered data to disk


In [ ]:
TRAIN_MODELS = 1
#
# CONFIGS
#
VERBOSE   = 2
SEED      = 42
NUM_FOLDS = 5

EPOCHS     = 8
BATCH_SIZE = 512

TARGET_LABEL      = 'target'
CUSTOMER_ID_LABEL = "customer_ID"

CATS = 11
NUMS_NO_E = 0
NUMS_WITH_E = 354
NUM_FEATURES = CATS + NUMS_NO_E + NUMS_WITH_E 

#NUM_FEATURES = 226



def build_gru_model():
    
    # INPUT - FIRST 11 COLUMNS ARE CAT, NEXT 177 ARE NUMERIC
    inp = tf.keras.Input(shape=(13,NUM_FEATURES))
    
    # input org is {CATS, NUM_NO_E, NUMS_WITH_E}
    
    # Categorical embeddings
    embeddings = []
    for k in range(11):
        emb = tf.keras.layers.Embedding(10,4)
        embeddings.append( emb(inp[:,:,k]) )
    
    # NaN embeddings
    ips = []
    for k in range (( CATS + NUMS_NO_E), NUM_FEATURES, 2):
        l = tf.keras.layers.Dense(1,activation = 'relu')
        ips.append(l(inp[:,:,k:k+2]))
        
    # x = tf.keras.layers.Concatenate()([inp[:,:,11:]]+embeddings)
    # x = tf.keras.layers.Concatenate()([inp[:,:,CATS:(CATS+NUMS_NO_E)]] + embeddings + ips)
    x = tf.keras.layers.Concatenate()(embeddings + ips)
    
    # SIMPLE RNN BACKBONE
    x = tf.keras.layers.GRU(units=256, return_sequences=False)(x)
    x = tf.keras.layers.Dense(64,activation='relu')(x)
    x = tf.keras.layers.Dense(32,activation='relu')(x)
    
    # OUTPUT
    x = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    
    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.BinaryCrossentropy()
    model.compile(loss=loss, optimizer = opt)
    
    return model

# CUSTOM LEARNING SCHEUDLE
import math
TOTAL_EPOCHS = EPOCHS

# CUSTOM LEARNING SCHEUDLE
def lrfn(epoch):
    lr = [1e-3]*5 + [1e-4]*2 + [1e-5]*1
    i = math.floor(len(lr) * (epoch/TOTAL_EPOCHS))
    return lr[i]
LR = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)


def extract_X_Y(FE_data, cids):
    #utils.pt(str(FE_data.shape))
    #utils.pt(str(cids.shape))
    
    data = FE_data.loc[(FE_data.customer_ID.isin(cids.customer_ID.values))]
    data = data.reset_index() # this adds "index" column to the data-frame
    
    data.info(memory_usage='deep')
    
    #utils.pt(str(data.shape))
    #data.info()
    #utils.pt(str(data.iloc[:,1:-1].values.shape))
    #utils.pt(str(data.columns))
    
    Y = data[['customer_ID','target']].drop_duplicates().sort_index().target.values
    utils.pt('Reshaping data')
    X = data.iloc[:,2:-1].values.reshape((-1,13, NUM_FEATURES))
    
    #FE_data.drop((FE_data.customer_ID.isin(cids.customer_ID.values)).index, inplace = True)
    
    return (X,Y)

def extract_val_train_data(fold, train_FE_data):
    train_cids = pd.read_feather(f'{PATH_TO_PROCESSED4_DATA}/train_{CUSTOMER_ID_LABEL}_fold_{fold}.f')
    val_cids   = pd.read_feather(f'{PATH_TO_PROCESSED4_DATA}/val_{CUSTOMER_ID_LABEL}_fold_{fold}.f')
    
    (X_train, Y_train) = extract_X_Y(train_FE_data, train_cids)
    (X_val  , Y_val  ) = extract_X_Y(train_FE_data, val_cids  )
    
    return (X_train, Y_train, X_val, Y_val)

def extract_test_data(train_FE_data):
    test_cids = pd.read_feather(f'{PATH_TO_PROCESSED4_DATA}/test_{CUSTOMER_ID_LABEL}.f')
    return (extract_X_Y(train_FE_data, test_cids))
    
    
def train_gru_models():
    
    utils.pt(f'Reading feature engineered data.')
#    train_FE_data = pd.read_feather(FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_FEATHER)
    
#     utils.pt(f'Extracting test data.')
    
#     (X_test, Y_test) = extract_test_data(train_FE_data)

#     utils.pt(f'### Test data shapes   {X_test.shape} , {Y_test.shape}')
    
    for fold in range(0,NUM_FOLDS):
        
        train_FE_data = pd.read_feather(FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_FEATHER)
        
        utils.pt(f'#### Fold : {fold} ####')
        
        utils.pt(f'Extracting train and val data.')
        (X_train, Y_train, X_val, Y_val) = extract_val_train_data(fold, train_FE_data)
        
        #utils.gc_l([train_FE_data])
        
        utils.pt(f'### Training data shapes   {X_train.shape} , {Y_train.shape}')
        utils.pt(f'### Validation data shapes {X_val.shape}   , {Y_val.shape}  ')
        
        
        utils.pt(f'Starting model training.')
        # BUILD AND TRAIN MODEL
        K.clear_session()
        model = build_gru_model()
        h = model.fit(X_train,Y_train, 
                      validation_data = (X_val,Y_val),
                      batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE,
                      callbacks = [LR])        
        utils.pt(f'Completed model training.')
        
        utils.pt(f'Saving model to file.')
        model.save_weights(f'{PATH_TO_MODEL}gru_fold_{fold}.h5')
        
        # INFER VALID DATA
        utils.pt('Inferring validation data...')
        preds = model.predict(X_val, batch_size=512, verbose=VERBOSE).flatten()
        amm = utils.amex_metric_mod(Y_val, preds)
        utils.pt(f'Fold {fold} CV = {amm}')
        
        
        # INFER TEST DATA
#         utils.pt('Inferring test data...')
#         preds = model.predict(X_test, batch_size=512, verbose=VERBOSE).flatten()
#         amm = utils.amex_metric_mod(Y_test, preds)
#         utils.pt(f'Fold {fold} CV = {amm}')
        
        print()
        
        utils.gc_l([model, X_train, Y_train, X_val, Y_val, preds, train_FE_data, amm, h])
        K.clear_session()
        
#        train_FE_data = pd.read_feather(FILENAME_TRAIN_PROCESSED_GRU_NAN_EMBEDDINGS_FEATHER)

    print()
    utils.pt(f'*** Completed model training for all folds ***')

if TRAIN_MODELS :
    train_gru_models()

In [4]:
for i in range(1,10, 2):
    print(i)

1
3
5
7
9


In [ ]:
def get_rows(customers, train, NUM_FOLDS = 10, verbose = ''):
    chunk = len(customers)//NUM_FOLDS
    if verbose != '':
        utils.pt(f'We will split {verbose} data into {NUM_FOLDS} separate folds.')
        utils.pt(f'There will be {chunk} customers in each fold (except the last fold).')
        utils.pt('Below are number of rows in each fold:')
    rows = []

    for k in range(NUM_FOLDS):
        if k==NUM_FOLDS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = train.loc[train.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': utils.pt( str(rows) )
    return rows

def get_column_names_cids(read_customer_hashes):
    
    # GET TEST COLUMN NAMES
    test = pd.read_csv(FILENAME_TEST_DATA_CSV, nrows=1)
    T_COLS = test.columns
    utils.pt(f'There are {len(T_COLS)} test dataframe columns')
    
    if read_customer_hashes:
        test_cids = pd.read_parquet(FILENAME_TEST_CUSTOMER_HASHES)
    else:
        test_cids = pd.read_csv(FILENAME_TEST_DATA_CSV, usecols=['customer_ID'])
        test_cids.to_parquet(FILENAME_TEST_CUSTOMER_HASHES)
    
    test_cids['customer_ID'] = test_cids['customer_ID'].str[-16:].apply(lambda x:int(x,16)).astype('int64')
    cids = test_cids.drop_duplicates().sort_index().values.flatten()
    
    utils.pt(f'There are {len(cids)} unique customers in test.')
    
    return (T_COLS, cids, test_cids)

def break_up_test_data(T_COLS, NUM_FILES, rows):
    
    # SAVE TEST CUSTOMERS INDEX
    test_customer_hashes = np.array([],dtype='int64')
    
    # CREATE PROCESSED TEST FILES AND SAVE TO DISK
    for k in range(NUM_FILES):

        # READ CHUNK OF TEST CSV FILE
        skip = int(np.sum( rows[:k] ) + 1) #the plus one is for skipping header
        test = pd.read_csv(FILENAME_TEST_DATA_CSV, nrows=rows[k], 
                              skiprows=skip, header=None, names=T_COLS)

        # FEATURE ENGINEER DATAFRAME
        test = feature_engineer(test, targets = None, edit_cid_time = True)
        
        # SAVE TEST CUSTOMERS INDEX
        cust = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
        test_customer_hashes = np.concatenate([test_customer_hashes,cust])
        
        # SAVE FILES
        utils.pt(f'Test_File_{k+1} has {test.customer_ID.nunique()} customers and shape {test.shape}')
        data = test.iloc[:,1:].values.reshape((-1,13,188))
        np.save(f'{PATH_TO_PROCESSED2_DATA}test_data_{k+1}',data.astype('float32'))
        
        # CLEAN MEMORY
        utils.gc_l([test, data])
    
    # SAVE CUSTOMER INDEX OF ALL TEST FILES
    np.save(FILENAME_TEST_HASH_DATA, test_customer_hashes)   

In [ ]:
PROCESS_TEST_DATA = 0
NUM_TEST_FILES = 20
READ_CUSTOMER_HASHES = 1

if PROCESS_TEST_DATA:
    (T_COLS,cids, test_cids) = get_column_names_cids(READ_CUSTOMER_HASHES)
    
    rows = get_rows(cids, test_cids, NUM_FOLDS = NUM_TEST_FILES, verbose = 'test')
    
    break_up_test_data(T_COLS, NUM_TEST_FILES, rows)

In [ ]:
INFER_TEST_DATA = 1

def infer_test_data():
    # INFER TEST DATA
    start = 0; end = 0
    sub = pd.read_csv(FILENAME_SAMPLE_SUBMISSION_CSV)
    
    # REARANGE SUB ROWS TO MATCH PROCESSED TEST FILES
    sub['hash'] = sub['customer_ID'].str[-16:].apply(lambda x:int(x,16)).astype('int64')
    test_hash_index = np.load(f'{FILENAME_TEST_HASH_DATA}.npy')
    sub = sub.set_index('hash').loc[test_hash_index].reset_index(drop=True)
    
    for k in range(NUM_TEST_FILES):
        # BUILD MODEL
        K.clear_session()
        model = build_gru_model()
        
        # LOAD TEST DATA
        utils.pt(f'Inferring Test_File_{k+1}')
        X_test = np.load(f'{PATH_TO_PROCESSED2_DATA}test_data_{k+1}.npy')
        end = start + X_test.shape[0]

        # INFER 5 FOLD MODELS
        model.load_weights(f'{PATH_TO_MODEL}gru_fold_0.h5')
        p = model.predict(X_test, batch_size=512, verbose=0).flatten() 
        for j in range(1,5):
            model.load_weights(f'{PATH_TO_MODEL}gru_fold_{j}.h5')
            p += model.predict(X_test, batch_size=512, verbose=0).flatten()
        p /= 5.0

        # SAVE TEST PREDICTIONS
        sub.loc[start:end-1,'prediction'] = p
        start = end
        
        # CLEAN MEMORY
        utils.gc_l([ model, X_test, p])
     
    sub.to_csv(FILENAME_GRU_SUBMISSION,index=False)
    print('Submission file shape is', sub.shape )
    display( sub.head() )

    
if INFER_TEST_DATA:
    infer_test_data()